In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/lacuna-malaria-detection-dataset/Train.csv')
unique_classes = df['class'].unique()
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
print(class_mapping)

{'Trophozoite': 0, 'NEG': 1, 'WBC': 2}


In [ ]:
!pip install -q ultralytics
!pip install -q tqdm

# Import libraries
# import wandb
# from wandb.integration.ultralytics import add_wandb_callback
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing

In [3]:
# Set the data directory
DATA_DIR = Path('/kaggle/input/lacuna-malaria-detection-dataset')

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Add an image_path column
train['image_path'] = [Path('/kaggle/input/lacuna-malaria-detection-dataset/images/' + x) for x in train.Image_ID]
test['image_path'] = [Path('/kaggle/input/lacuna-malaria-detection-dataset/images/' + x) for x in test.Image_ID]


# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map({'Trophozoite': 0, 'NEG': 1, 'WBC': 2})

# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset=['Image_ID'], ignore_index=True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size=0.1, stratify=train_unique_imgs_df['class'], random_state=42)

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check the shapes of training and validation data
print(X_train.shape, X_val.shape)

# Define directories for images and labels
TRAIN_IMAGES_DIR = Path('/kaggle/working/train/images')
VAL_IMAGES_DIR = Path('/kaggle/working/val/images')
TEST_IMAGES_DIR = Path('/kaggle/working/test/images')
TRAIN_LABELS_DIR = Path('/kaggle/working/train/labels')
VAL_LABELS_DIR = Path('/kaggle/working/val/labels')
TEST_LABELS_DIR = Path('/kaggle/working/test/labels')

# Create necessary directories
for DIR in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_LABELS_DIR]:
    if DIR.exists():
        shutil.rmtree(DIR)
    DIR.mkdir(parents=True, exist_ok=True)

# Copy train, val, and test images to their respective dirs
for img in tqdm(X_train.image_path.unique()):
    shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])

for img in tqdm(X_val.image_path.unique()):
    shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])

for img in tqdm(test.image_path.unique()):
    shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])

# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):
    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"

    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    with multiprocessing.Pool() as pool:
        list(tqdm(pool.imap(save_yolo_annotation, dataframe.to_dict('records')), total=len(dataframe)))

# Save train and validation labels to their respective dirs
process_dataset(X_train, TRAIN_LABELS_DIR)
process_dataset(X_val, VAL_LABELS_DIR)

# Create a data.yaml file required by YOLO
class_names = train['class'].unique().tolist()
num_classes = len(class_names)

data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names
}

# Save the data.yaml file
yaml_path = Path('data.yaml')
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)


(21260, 9) (2270, 9)


  0%|          | 0/2472 [00:00<?, ?it/s]

  0%|          | 0/275 [00:00<?, ?it/s]

  0%|          | 0/1178 [00:00<?, ?it/s]

/tmp/ipykernel_30/3331563916.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/21260 [00:00<?, ?it/s]

/tmp/ipykernel_30/3331563916.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/2270 [00:00<?, ?it/s]

In [5]:
# Load a YOLO pretrained model
model = YOLO("yolo11s.pt")

model.train(
    data='data.yaml',          
    imgsz=640,
    batch=64,
    optimizer='SGD',
    rect=False,
    lr0=0.01,
    cos_lr=True,
    freeze=None,
    device=[0,1],
    name='exp17'
)

Ultralytics 8.3.29 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=exp17, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

2024-11-13 09:59:04,774	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-13 09:59:05,304	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning /kaggle/working/train/labels.cache... 2472 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2472/2472 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/val/labels.cache... 275 images, 0 backgrounds, 0 corrupt: 100%|██████████| 275/275 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        275       2270      0.508      0.544      0.551      0.285

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.65G       1.53     0.7743     0.9225        184        640: 100%|██████████| 39/39 [01:11<00:00,  1.84s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.43it/s]


                   all        275       2270      0.514       0.54      0.551      0.284

100 epochs completed in 2.254 hours.
Optimizer stripped from runs/detect/exp17/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/exp17/weights/best.pt, 19.2MB

Validating runs/detect/exp17/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:10<00:00,  2.06s/it]


                   all        275       2270      0.524      0.529      0.552      0.286
           Trophozoite        202       1525       0.65      0.655      0.684       0.27
                   NEG         69         69          0          0          0          0
                   WBC        164        676      0.923      0.932      0.973      0.587
Speed: 0.1ms preprocess, 3.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/exp17


In [6]:
model = YOLO('/kaggle/working/runs/detect/exp17/weights/best.pt')

In [ ]:
TEST_IMAGES_DIR = Path('/kaggle/working/test/images')

In [8]:
# Habilitar el modo evaluación
model.val()

# Hacer predicciones en imágenes de prueba y guardarlas en un archivo CSV
image_files = os.listdir(TEST_IMAGES_DIR)
all_data = []
neg_prediction = 0  

for image_file in tqdm(image_files):
    img_path = os.path.join(TEST_IMAGES_DIR, image_file)
    results = model(img_path, verbose=False)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    confidences = results[0].boxes.conf.tolist()
    names = results[0].names

    if not boxes:
        neg_prediction += 1
        all_data.append({
            'Image_ID': image_file,
            'class': 'NEG',
            'confidence': 1.0,
            'ymin': 0,
            'xmin': 0,
            'ymax': 0,
            'xmax': 0
        })
    else:
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[int(cls)]

            all_data.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })

print(f'========== Total NEG Prediction {neg_prediction} ================================')

# Convertir los resultados a un DataFrame y guardarlos
sub = pd.DataFrame(all_data)
sub.to_csv('/kaggle/working/yolo11_exp17_submission.csv', index=False)

Ultralytics 8.3.29 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /kaggle/working/val/labels.cache... 275 images, 0 backgrounds, 0 corrupt: 100%|██████████| 275/275 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:10<00:00,  1.70it/s]


                   all        275       2270      0.524      0.526      0.551      0.286
           Trophozoite        202       1525      0.651      0.648      0.682      0.271
                   NEG         69         69          0          0          0          0
                   WBC        164        676      0.923      0.929      0.973      0.587
Speed: 0.2ms preprocess, 6.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val16


  0%|          | 0/1178 [00:00<?, ?it/s]

========== Total NEG Prediction 300 ================================


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
